In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import glob
import os
from obspy import read, UTCDateTime, read_inventory
from obspy.signal import PPSD
import warnings
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib as mpl
plt.style.use("ggplot")

from msnoise.api import *
from wxs_dvv import *

plt.rcParams['figure.figsize'] = (16,8)

# Compute dv/v using the wavelet method

Following the excercise exploring the classical (MWCS) processing of continuous seismic records for monitoring using seismic interferometry with MSNoise, we now explore a different way to measure relative variations of seismic velocity from cross-correlation functions (CCFs). We just explored increasing the the temporal resolution with sub-daily processing, we are now exploring increasing the spectral resolution and its benefits.

Shujuan Mao's paper

The Wavelet method presents particular benefit when monitoring volcanoes as it provides a proxy for the depth of one, or several potential sources of pressurization under a volcano which, in turn, can be critical to forecasting an eruption or understanding its co-eruptive behavior.

- Here we will try and reproduce the dv/v measurements for the same 3 stations of the Piton de la Fournaise volcano, picking up after the stacking step. Here we are using the wavelet method, meaning that we can easily adjust the frequeny range of interest after processing the CCFs
- While you do this exercise, think about the advatange this method can have over the MWCS, but also its disadvantages.
- This provides also another template for how you can hack into MSNoise to develop you own methodologies, for monitoring or any other applications!

## Getting parameters from the msnoise DB

Again, when pluging into an MSNoise workflow, you can always use the parameters as they are in the database, or modify them below to explore the results (e.i. Start, end freqmin, freqmax, mov_stack,...).

In [ ]:
# connect to the database
db = connect()

# Obtain a list of dates between ``start_date`` and ``enddate``
start, end, datelist = build_movstack_datelist(db)

# Get the list of parameters from the DB:
params = get_params(db)

# Get the time axis for plotting the CCF:
taxis = get_t_axis(db)

filter_id = 5
comp = "EN"

station= "PF.CSS.00"

pair="{}:{}".format(station,station)

# Get the results for two station, filter id=1, ZZ component, mov_stack=("1d","1d") and the results as a 2D array:
ccfs = get_results_all(db, station, station, filter_id, comp, datelist, format="xarray")

# Reading the CCFs
Let's first take a look at an individual trace and it's corresponding reference trace

In [ ]:
ref=ccfs.CCF.median(axis=0).data
cur_trace = ccfs.CCF[955]
current = cur_trace.data
t = ccfs.CCF.mean(axis=0).taxis.data
# We normalise the traces here
ori_waveform = (ref/ref.max())
new_waveform = (current/current.max())

Go ahead and plot it! have fun with the figure too!

In [ ]:
fig = plt.figure()
plt.plot(t,new_waveform, label="individual trace", alpha=0.7)
plt.plot(t,ori_waveform, label="Reference")

plt.xlim(-50,50)
plt.legend()
plt.show()

In [ ]:
filters = get_filters(db, all=False)

# We areworking with the broad-ish frequency range of the filter 5
freqmin=filters[0].low
freqmax=filters[0].high
fs = params.cc_sampling_rate
lag_min = params.dtt_minlag
lag_max = params.dtt_minlag+params.dtt_width

date = cur_trace.times.data # The time of the current trace

## The cross-wavelet transform
Now we use the same function described in Shujuan's paper on both traces

The inputs are the following:

    trace_ref,
    trace_current,
    fs, Sampling frequency --> extracted from the DB
    ns, smoothing parameter
    nt, smoothing parameter
    vpo, Spacing parameter between discrete scales, higher means finer resolution
    freqmin,
    freqmax,
    nptsfreq, Number of frequency points between freqmin and freqmax

In [ ]:
# Cross wavelet transform
WXamp, WXspec, WXangle, Wcoh, WXdt, freqs, coi = xwt(ori_waveform, new_waveform, fs, 3, 0.25, 10, freqmin, freqmax, 100)

While most of the magic happened in the cell above, we still don't have a dv/v.
For this we will calculate a similar linear regression as the one discussed yesterday for the MWCS method. Here, however, we calculate it for every frequency point and using a weighting function rejecting data point with low coherence

In [ ]:
# get the dv/v from the linear regression and the weighting function
dvv, err, wf =get_dvv(freqs, t, WXamp, Wcoh, WXdt, lag_min, lag_max, freqmin=freqmin, freqmax=freqmax)

In [ ]:
#Plotting the results
do_plot(t, WXamp, WXspec, WXangle, Wcoh, WXdt, freqs, coi, wf, pair, date, comp)

Now you will find the figure in "WCT/Figure", let's talk about what it means...

# Going bigger
Now let's run the job for all dates and all components available. This might take a while, so start with a subset of dates in the time_range variable. Try something like `time_range = ccfs.times.data[910:960]` to work on only 50 time steps at the time. With enough time, try to look at the output for all 3 stations.


In [ ]:
plot = False
saveit=False
import tqdm
comps=["EN", "EZ", "NZ"]
fig, axes = plt.subplots(len(comps),1, figsize=(10,10))
filterid = 5
taxis = get_t_axis(db)
mov_stack = params.mov_stack[-1]
s1 = 'PF.FOR.00'
s2 = s1

iax=0

plt.suptitle(mov_stack)
filter = get_filters(db, ref=filterid)
for i, comp in enumerate(comps):
    plt.sca(axes[iax])      
    try:
        ref = xr_get_ref(s1, s2, comp, filterid, taxis)
        ref = ref.CCF.values
        ori_waveform = (ref/ref.max()) #TODO make normalisation optional
    except FileNotFoundError as fullpath:
        print("FILE DOES NOT EXIST: %s, skipping" % fullpath)
        continue
    if not len(ref):
        continue                
    dvv_list = []
    err_list = []
    data_dates = []
    try:
        ccfs = get_results_all(db, s1, s2, filter_id, comp, datelist, format="xarray")
    except FileNotFoundError as fullpath:
        print("FILE DOES NOT EXIST: %s, skipping" % fullpath)
        continue
    print("Processing %s:%s f%i m%s %s" % (s1, s2, filterid, mov_stack, comp))
    ### TIME RANGE can be edited here!
    time_range = ccfs.times.data[910:960]
    pbar = tqdm.tqdm(time_range, desc="looping through dates")
    for d, date in enumerate(pbar):        

        pbar.set_description("looping through dates {}".format(date))
        current = ccfs.CCF[d].values
        new_waveform = (current/current.max())
        
        WXamp, WXspec, WXangle, Wcoh, WXdt, freqs, coi = xwt(ori_waveform, new_waveform, fs, 3, 0.25, 10, freqmin, freqmax, 400)# TODO get freq lims from db 
        dvv, err, wf =get_dvv(freqs, t, WXamp, Wcoh, WXdt, lag_min, lag_max, freqmin=freqmin, freqmax=freqmax)
        dvv_list.append(dvv)
        err_list.append(err)
        data_dates.append(date)
        if plot:
            do_plot(t, WXamp, WXspec, WXangle, Wcoh, WXdt, freqs, coi, wf, pair, date, comp)
#                    del dvv, err

    if len(dvv_list)>1: # Check if the list has more than 1 measurement to save it
        dvv_df = pd.DataFrame(columns=freqs, index=data_dates)
        err_df = pd.DataFrame(columns=freqs, index=data_dates)
        pbar = tqdm.tqdm(data_dates, desc="Formating the DataFrame")
        for i, date in enumerate(pbar):
            dvv_df.iloc[i]=dvv_list[i]
            err_df.iloc[i]=err_list[i]
        if saveit:
            if not os.path.isdir("WCT"):
                os.makedirs("WCT")
            dfn = "{} {}_ {} - {}.pkl".format(pair.replace(":","_"),comp,str(dvv_df.index[0].date()),str(dvv_df.index[-1].date()))
            efn = "Err {} {}_ {} - {}.pkl".format(pair.replace(":","_"),comp,str(dvv_df.index[0].date()),str(dvv_df.index[-1].date()))
            path = os.path.join("WCT",dfn)
            epath = os.path.join("WCT",efn)
            dvv_df.to_pickle(path)    # Save dvv
            err_df.to_pickle(epath)


    clim = 2
    span = 30 # Smoothing alert!
    axes[iax].pcolormesh(np.asarray(dvv_df.ewm(span = span).mean().index), 
                   np.asarray(dvv_df.ewm(span = span).mean().columns), 
                   dvv_df.ewm(span = span).mean().astype(float).T.values, 
                   cmap='seismic_r', edgecolors='none', vmin=-clim, vmax=clim)
    axes[iax].set_ylim(0.5, 2)
    cmap = mpl.cm.seismic_r
    norm = mpl.colors.Normalize(vmin=-clim, vmax=clim)
    cbar2=plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax = axes[iax])
    cbar2.set_label('dv/v', rotation=270)
    plt.title("Filter %i (%.2f - %.2f Hz) - %s" % (filterid, 0.5, 2, comp))            
    plt.axvline(datetime.date(2014,6,21), c='r', ls='--')
    plt.legend()
    plt.xlabel("")
    plt.ylabel("Frequency (Hz)")
    iax+=1
plt.tight_layout()
print("All done!")

When you manage to process the whole period, you are looking at the dv/v between 0.5 and 2 Hz at a high spectral resolution. Take some time to think about what you are looking at.

If you have extra time on your hands, try and average ths wavelet plot across the same frequency ranges in the filters precessed in the first notebook to campare the result with the one preduced with the MWCS method.